In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
print(os.listdir("/content/drive/My Drive/objects"))
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=255
IMAGE_HEIGHT=255
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [ ]:
filenames = os.listdir("/content/drive/My Drive/objects/training")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category[0] == 'b':
        categories.append(0)
    elif category[0] == 'c':
        categories.append(1)
    elif category[0]=='l':
        categories.append(2)
    else:
        categories.append(3)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
df['category'].value_counts()

In [ ]:
sample = random.choice(filenames)
image = load_img("/content/drive/My Drive/objects/training/"+sample)
plt.imshow(image)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
vgg16=VGG16()

In [ ]:
vgg16.summary()

In [ ]:
x=Flatten()(vgg16.output)

In [ ]:
vgg16_model = VGG16()
model = Sequential()

for layer in vgg16_model.layers[:-1]: # this is where I changed your code
    model.add(layer)    

# Freeze the layers 
for layer in model.layers:
    layer.trainable = False

# Add 'softmax' instead of earlier 'prediction' layer.
model.add(Dense(4, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
for layer in vgg16.layers:
    layer.trainable=False

In [ ]:
x=Dense(512, activation='relu')(vgg16.output)
#x=BatchNormalization()(x)
output = Dense(4, activation='softmax')(x)


In [ ]:
model=Model(inputs=vgg16.input,outputs=output)

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
earlystop = EarlyStopping(patience=10)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
df["category"] = df["category"].replace({0: 'bike',1: 'car',2:'laptop',3:'mobile'}) 

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    r"/content/drive/My Drive/objects/training", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
train_df.shape

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    r"/content/drive/My Drive/objects/training", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    r"/content/drive/My Drive/objects/training/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical'
)

# New Section

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
epochs=3 if FAST_RUN else 50
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)
#    callbacks=callbacks


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
img = image.load_img(r"/content/drive/My Drive/objects/training/bikes201.jpg", target_size=(255, 255))

# Preprocessing the image
x = image.img_to_array(img)
# x = np.true_divide(x, 255)
x = np.expand_dims(x, axis=0)

# Be careful how your trained model deals with the input
# otherwise, it won't make correct prediction!
x = preprocess_input(x, mode='caffe')

preds = model.predict(x)
print(preds)

In [ ]:
test_filenames = os.listdir("/content/drive/My Drive/objects/testing")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    r"/content/drive/My Drive/objects/testing/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False)

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))


In [ ]:
test_df['category']=np.argmax(predict, axis=-1)

In [ ]:
test_df

In [ ]:
test_df.iloc[50:80,:]

In [ ]:
model.save(r"/content/drive/My Drive/objects/new_model_98.h5")

In [ ]:
from keras.models import load_model
 
# load model
model_98 = load_model(r'/content/drive/My Drive/objects/new_model_98.h5')
# summarize model.
model_98.summary()

In [ ]:
filenames = os.listdir("/content/drive/My Drive/objects/testing")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category[0] == 'b':
        categories.append(0)
    elif category[0] == 'c':
        categories.append(1)
    elif category[0]=='l':
        categories.append(2)
    else:
        categories.append(3)

df_testing = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
df_testing

In [ ]:
test_df

In [ ]:
confusion_matrix(df_testing['category'].to_list(),test_df['category'].to_list())

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
import keras
keras.__version__

In [ ]:
import PIL

In [ ]:
PIL.__version__